# Demo: Transform a Label Image from SmartSPIM to CCF Space

## Overview

This notebook demonstrates how to apply the transformation results from the `RegisterToCCF` reproducible pipeline notebook to other data initially in correspondence with the input SmartSPIM image. The notebook can be run interactively in CodeOcean.

## Inputs

1. Source label image in correspondence with the input SmartSPIM image used in the `RegisterToCCF` notebook.

2. Transform stack results from `RegisterToCCF` notebook. This is a sequence of transforms mapping from SmartSPIM sample space to CCF atlas$^1$ space produced by the SmartSPIM to CCF registration pipeline.
  - A translation-only ITK transform coarsely aligns the label image to the CCF atlas;
  - Three Elastix transform stages deformably map from initialized to CCF space.

## Outputs

1. Source image, mesh, or point set resampled into CCF space.

## Assumptions

1. Source data initially exists in spatial alignment with the original stitched SmartSPIM image input. This can be confirmed with a spatial viewer such as ITKWidgets, Neuroglancer, or 3D Slicer.

## Procedure

1. Data is read in from their respective stores attached via CodeOcean's data attachment mechanism.

2. An initial translation is applied to coarsely superimpose the label image on the CCF target image space. The source image is updated in place without resampling.

3. The initialized label image is aligned to the target CCF image space. ITKElastix is used to apply three transform stages for rigid, affine, and then deformable registration.

4. Results are written out.

![Label Map Before and After Transformation](images/transform-label-map.png)

## References

1. Quanxin Wang, Song-Lin Ding, Yang Li, Josh Royall, David Feng, Phil Lesnar, Nile Graddis, Maitham Naeemi, Benjamin Facer, Anh Ho, Tim Dolbeare, Brandon Blanchard, Nick Dee, Wayne Wakeman, Karla E. Hirokawa, Aaron Szafer, Susan M. Sunkin, Seung Wook Oh, Amy Bernard, John W. Phillips, Michael Hawrylycz, Christof Koch, Hongkui Zeng, Julie A. Harris, Lydia Ng,
The Allen Mouse Brain Common Coordinate Framework: A 3D Reference Atlas, Cell, Volume 181, Issue 4, 2020, Pages 936-953.e20, ISSN 0092-8674, https://doi.org/10.1016/j.cell.2020.04.007

## Initialize the Notebook

In [1]:
import os
import json
from pathlib import Path

import numpy as np
import itk
import itkwidgets

In [2]:
SOURCE_IMAGE_INPUT_FILEPATH = "../data/SmartSPIM_631680_2022-09-09_13-52-33_stitched_2022-11-10_17-18-18/processed/OMEZarr/Ex_647_Em_690.zarr"
SAMPLE_ID = int(SOURCE_IMAGE_INPUT_FILEPATH.split("_")[1])
SAMPLE_CHANNEL = SOURCE_IMAGE_INPUT_FILEPATH.split("/")[-1].split(".zarr")[0]
SAMPLE_LEVEL = (
    4  # data is scaled down by 2 ^ N in each direction or by 2 ^ 3N in total volume
)
SAMPLE_NAME = f"{SAMPLE_ID}_{SAMPLE_CHANNEL}"

LABEL_IMAGE_INPUT_FILEPATH = "../data/demo_annotations_631680/631680_Caudoputamen.nrrd"
LABEL_VALUE = 1 # Identify the label region of interest in the label image

# Also available at http://download.alleninstitute.org/informatics-archive/converted_mouse_ccf/average_template/
TARGET_IMAGE_INPUT_FILEPATH = (
    "../data/allen_mouse_ccf/average_template/average_template_25.nii.gz"
)

TRANSFORMS_PATH = f"../data/demo_registration_results_631680"
N_ELASTIX_STAGES = 3
ITK_TRANSFORM_FILENAME = f"{TRANSFORMS_PATH}/init-transform.hdf5"
ELASTIX_TRANSFORM_FILENAMES = [
    Path(f"{TRANSFORMS_PATH}/elastix-transform{index}.txt")
    for index in range(N_ELASTIX_STAGES)
]

RESULTS_PATH = f"../results/{SAMPLE_NAME}"
LABEL_IMAGE_OUTPUT_FILEPATH = (
    f"{RESULTS_PATH}/{Path(LABEL_IMAGE_INPUT_FILEPATH).stem}_transformed.nii.gz"
)

print(f"Transform results will be written to {LABEL_IMAGE_OUTPUT_FILEPATH}")

Transform results will be written to ../results/631680_Ex_647_Em_690/631680_Caudoputamen_transformed.nii.gz


## Load Transforms

The `RegisterToCCF` notebook outputs a series of transforms:

1. An ITK transform denoting an initial translation for alignment;
2. Three Elastix transforms denoting rigid, affine, and deformable registration steps.

In [3]:
assert os.path.exists(
    ITK_TRANSFORM_FILENAME
), f"Could not find transforms in {TRANSFORMS_PATH} (did you run the registration pipeline first?)"

init_transform = itk.transformread(ITK_TRANSFORM_FILENAME)[0]
print(init_transform)

VersorRigid3DTransform (0x55b68f650210)
  RTTI typeinfo:   itk::VersorRigid3DTransform<double>
  Reference Count: 1
  Modified Time: 536
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 0 0 
    0 1 0 
    0 0 1 
  Offset: [-12.3259, -2.6141, -8.1635]
  Center: [6.6384, 9.2016, 4.176]
  Translation: [-12.3259, -2.6141, -8.1635]
  Inverse: 
    1 0 0 
    0 1 0 
    0 0 1 
  Singular: 0
  Versor: [ 0, 0, 0, 1 ]



In [4]:
assert all([p.exists() for p in ELASTIX_TRANSFORM_FILENAMES])

toplevel_param = itk.ParameterObject.New()
param = itk.ParameterObject.New()

for index in range(N_ELASTIX_STAGES):
    param.ReadParameterFile(str(ELASTIX_TRANSFORM_FILENAMES[index]))
    print(type(param.GetParameterMap(0)))
    toplevel_param.AddParameterMap(param.GetParameterMap(0))

print(toplevel_param)

<class 'itk.elxParameterObjectPython.mapstringvectorstring'>
<class 'itk.elxParameterObjectPython.mapstringvectorstring'>
<class 'itk.elxParameterObjectPython.mapstringvectorstring'>
ParameterObject (0x55b691a0d220)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 74
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (CenterOfRotationPoint -5.6875 6.5875 -3.9875)
  (CompressResultImage "false")
  (ComputeZYX "false")
  (DefaultPixelValue 0)
  (Direction 0 1 0 0 0 -1 -1 0 0)
  (FinalBSplineInterpolationOrder 3)
  (FixedImageDimension 3)
  (FixedInternalImagePixelType "float")
  (HowToCombineTransforms "Compose")
  (Index 0 0 0)
  (InitialTransformParametersFileName "NoInitialTransform")
  (MovingImageDimension 3)
  (MovingInternalImagePixelType "float")
  (NumberOfParameters 6)
  (Origin 0 0 0)
  (ResampleInterpolator "FinalBSplineInterpolator")
  (Resampler "DefaultResampler")
  (ResultImageFormat "nii")
  (ResultImagePixelType "floa

## Load Annotated Label Image Data

We read in a label image that is in initial _spatial_ correspondence with the SmartSPIM moving image used in registration. The label image is specified as follows:

1. Voxels not in a region of interest are labelled with the background value 0;

2. Other regions of interest are labelled with discrete values (1, 2, ...)

To mitigate interpolation issues, only one label region may be mapped to CCF space at a given time. A binary image is constructed to reflect a specified label region, with other label values treated as background.

In [5]:
label_image = itk.imread("../data/demo_annotations_631680/631680_Caudoputamen.nrrd")

print(label_image)
print(f"Label values: {np.unique(label_image)}")

Image (0x55b692cd85d0)
  RTTI typeinfo:   itk::Image<unsigned char, 3u>
  Reference Count: 1
  Modified Time: 977
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 786
  UpdateMTime: 976
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [462, 640, 262]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [462, 640, 262]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [462, 640, 262]
  Spacing: [0.0288, 0.0288, 0.032]
  Origin: [13.2768, 18.4032, 8.352]
  Direction: 
-1 0 0
0 -1 0
0 0 -1

  IndexToPointMatrix: 
-0.0288 0 0
0 -0.0288 0
0 0 -0.032

  PointToIndexMatrix: 
-34.7222 0 0
0 -34.7222 0
0 0 -31.25

  Inverse Direction: 
-1 0 0
0 -1 0
0 0 -1

  PixelContainer: 
    ImportImageContainer (0x55b691a068b0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned lon

In [6]:
# Convert to binary image for a specified label region of interest

binary_image = itk.binary_threshold_image_filter(
    label_image,
    lower_threshold=LABEL_VALUE,
    upper_threshold=LABEL_VALUE,
    inside_value=1,
    outside_value=0,
)

print(f"{np.min(binary_image)} {np.max(binary_image)}")

0 1


## Apply Initial ITK Transform

First, we apply in-place translation with ITK by directly updating the image origin. See `RegisterToCCF` for more information.

In [7]:
# Given that the initial transform represents only a translation, we can update the origin accordingly in-place

change_information_filter = itk.ChangeInformationImageFilter[type(binary_image)].New()
change_information_filter.SetInput(binary_image)
change_information_filter.SetOutputOrigin(
    init_transform.TransformPoint(itk.origin(binary_image))
)
change_information_filter.ChangeOriginOn()
change_information_filter.UpdateOutputInformation()

binary_image_init = change_information_filter.GetOutput()
print(binary_image_init)

print(f"{np.min(binary_image_init)} {np.max(binary_image_init)}")
print(f"{np.unique(binary_image_init)}")

Image (0x55b692ac4ca0)
  RTTI typeinfo:   itk::Image<unsigned char, 3u>
  Reference Count: 2
  Modified Time: 1041
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x55b692ac2480) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 1035
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [462, 640, 262]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [0, 0, 0]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [0, 0, 0]
  Spacing: [0.0288, 0.0288, 0.032]
  Origin: [0.9509, 15.7891, 0.1885]
  Direction: 
-1 0 0
0 -1 0
0 0 -1

  IndexToPointMatrix: 
-0.0288 0 0
0 -0.0288 0
0 0 -0.032

  PointToIndexMatrix: 
-34.7222 0 0
0 -34.7222 0
0 0 -31.25

  Inverse Direction: 
-1 0 0
0 -1 0
0 0 -1

  PixelContainer: 
    ImportImageContainer (0x55b693d107f0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned lon

## Apply Deformable Alignment with `itk-elastix`

We subsequently align the initialized label volume to CCF atlas space with three stages of Elastix deformable transformations.

Transformix (Elastix) requires that input images have floating-point types. To avoid label interpolation issues we constrain our deformable B-spline transform stage to zero-order B-spline interpolation.

In [8]:
# Update interpolation order for binary image
toplevel_param.SetParameter("FinalBSplineInterpolationOrder", "0")

In [9]:
binary_image_f = itk.cast_image_filter(
    binary_image_init,
    ttype=[
        type(binary_image_init),
        itk.Image[itk.F, binary_image_init.GetImageDimension()],
    ],
)

binary_image_transformed = itk.transformix_filter(
    binary_image_f, transform_parameter_object=toplevel_param
)

print(
    f"Output data range: [{np.min(binary_image_transformed)},{np.max(binary_image_transformed)}]"
)

Output data range: [0.0,1.0]


In [10]:
# Rescale the binary image so that the label value matches that of the input image
binary_image_transformed = itk.cast_image_filter(
    binary_image_transformed, ttype=[type(binary_image_transformed), type(binary_image)]
)
label_image_transformed = itk.multiply_image_filter(
    binary_image_transformed, constant=int(LABEL_VALUE)
)

# Verify that only background and label values are present in the output image
print(f"Label values in transformed label image: {np.unique(label_image_transformed)}")

Label values in transformed label image: [0 1]


## Save Results

The transformed label image is saved in compressed NIFTI format for subsequent use.

The label image mapped from SmartSPIM to CCF space can be used in registration evaluation techniques, such as for overlap evaluation with dice coefficient evaluation against a corresponding CCF label.

In [11]:
os.makedirs(RESULTS_PATH, exist_ok=True)
itk.imwrite(label_image_transformed, LABEL_IMAGE_OUTPUT_FILEPATH, compression=True)